<a href="https://colab.research.google.com/github/Snazz2001/100-Days-Of-ML-Code/blob/master/vertex_ai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
import vertexai
# 初始化时填入你的项目 ID 和位置
vertexai.init(project="zinc-style-482013-i3", location="asia-south1")

In [8]:
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool
import vertexai

# Create a RAG Corpus, Import Files, and Generate a response

# TODO(developer): Update and un-comment below lines
PROJECT_ID = "zinc-style-482013-i3"
display_name = "factor_investing_corpus"
paths = ["gs://factor_investing_dxb"]
# paths = ["https://drive.google.com/file/d/123", "gs://my_bucket/my_files_dir"]  # Supports Google Cloud Storage and Google Drive Links

# Initialize Vertex AI API once per session
vertexai.init(project=PROJECT_ID, location="europe-west4")

# Create RagCorpus
# Configure embedding model, for example "text-embedding-005".
embedding_model_config = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005"
    )
)

rag_corpus = rag.create_corpus(
    display_name=display_name,
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=embedding_model_config
    ),
)

# Import Files to the RagCorpus
rag.import_files(
    rag_corpus.name,
    paths,
    # Optional
    transformation_config=rag.TransformationConfig(
        chunking_config=rag.ChunkingConfig(
            chunk_size=512,
            chunk_overlap=100,
        ),
    ),
    max_embedding_requests_per_min=1000,  # Optional
)

# Direct context retrieval
rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5),  # Optional
)
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="who wrote this book and why it is useful?",
    rag_retrieval_config=rag_retrieval_config,
)
print(response)

# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)

# Create a Gemini model instance
rag_model = GenerativeModel(
    model_name="gemini-2.0-flash", tools=[rag_retrieval_tool]
)

# Generate response
response = rag_model.generate_content("who wrote this book and why it is useful?")
print(response.text)
# Example response:
#   RAG stands for Retrieval-Augmented Generation.
#   It's a technique used in AI to enhance the quality of responses
# ...

contexts {
  contexts {
    source_uri: "gs://factor_investing_dxb/causal-factor-investing.pdf"
    text: "Marcos M. López de Prado\r\nADIA Lab\r\nhttps://doi.org/10.1017/9781009397315 Published online by Cambridge University PressShaftesbury Road, Cambridge CB2 8EA, United Kingdom\r\nOne Liberty Plaza, 20th Floor, New York, NY 10006, USA\r\n477 Williamstown Road, Port Melbourne, VIC 3207, Australia\r\n314–321, 3rd Floor, Plot 3, Splendor Forum, Jasola District Centre,\r\nNew Delhi – 110025, India\r\n103 Penang Road, #05–06/07, Visioncrest Commercial, Singapore 238467\r\nCambridge University Press is part of Cambridge University Press & Assessment,\r\na department of the University of Cambridge.\r\nWe share the University’s mission to contribute to society through the pursuit of\r\neducation, learning and research at the highest international levels of excellence.\r\nwww.cambridge.org\r\nInformation on this title: www.cambridge.org/9781009397292\r\nDOI: 10.1017/9781009397315\r\n© Marco

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


NotFound: 404 Publisher Model `projects/zinc-style-482013-i3/locations/asia-south1/publishers/google/models/gemini-2.0-flash-001` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions